In [1]:
# -*- coding: utf-8 -*-
import nltk
import json
import pandas as pd
import codecs

#### Pozitif ve negatif kelimeler pre-trained vektörler kullanılarak bulunulabilir.

In [2]:
keyword = 'yardım'
interested_words = ['sosyal','vakfı','vakıf','yoksul','muhtaç']
negative_words = ['hükümet','afganistan','bosna','yardımcı']
data = []
#stopwords = []

In [3]:
with codecs.open('items.jl', encoding="utf-8") as f:
	for line in f:
		data.append(json.loads(line))

'''with codecs.open('stopwords', encoding="utf-8") as f:
	for line in f:
		stopwords.append(line)

stopwords = [word.replace("\r\n","") for word in stopwords]'''

'with codecs.open(\'stopwords\', encoding="utf-8") as f:\n\tfor line in f:\n\t\tstopwords.append(line)\n\nstopwords = [word.replace("\r\n","") for word in stopwords]'

#### word_tokenize() fonksiyonu nltk'nin 'punkt' paketini kullanıyor.
#### Genelleştirmek için stopword'leri eliyebiliriz. Fakat bu keyword için hiçbir değişiklik olmadığı için comment'e aldım.

In [4]:
tokenized_sentences = [nltk.word_tokenize(x['name']) for x in data]

'''for tokenlist in tokenized_sentences:
	for token in tokenlist:
		if any(token==word for word in stopwords):
			del token'''

'for tokenlist in tokenized_sentences:\n\tfor token in tokenlist:\n\t\tif any(token==word for word in stopwords):\n\t\t\tdel token'

#### Eğer pozitif kelimeden sonra asıl keyword'ümüz geliyosa weight'i fazla oluyor
#### Genelleştirmek için; pozitif kelimelerin weight'i keyword'e yakınlığı üzerinden hesaplanabilir.

In [5]:
for i, tokenlist in enumerate(tokenized_sentences):
	weight = 0
	for j, text in enumerate(tokenlist):
		text = text.lower()
		if any(word in text for word in negative_words):
			continue
		elif text.find(keyword) != -1:
			weight += 10
		elif any(word in text for word in interested_words):
			tokenlist[j+1] = tokenlist[j+1].lower()
			if j != len(tokenlist)-1 and tokenlist[j+1].find(keyword) != -1:
				weight += 10
			else:
				weight += 2
	data[i]['weight'] = weight

#### Veriyi baştan dataframe'e atayarak weight hesaplamayı denedim. Daha fazla zaman alıyor.
#### Ama sort işlemi gayet hızlı.

In [6]:
sorted_data = pd.DataFrame(data).sort_values(by='weight', ascending=False)
with codecs.open('results.jl', 'w', encoding='utf-8') as file:
	sorted_data.to_json(file, orient='records', lines=True, force_ascii=False)

In [7]:
sorted_data.head(10)

,links,name,weight
112,[{'link': 'https://www.tbmm.gov.tr/tutanaklar/...,Bursa-Orhaneli köylerine Sosyal Yardımlaşma ve...,90
86,[{'link': 'https://www.tbmm.gov.tr/tutanaklar/...,"Ardahan Milletvekili Ensar Öğüt’ün, bazı sınır...",40
87,[{'link': 'https://www.tbmm.gov.tr/tutanaklar/...,"Belediyelerin sosyal yardımlarına (6/191), Bel...",40
0,[{'link': 'https://www.tbmm.gov.tr/tutanaklar/...,Adıyaman'da Sosyal Yardımlaşma ve Dayanışma Va...,34
150,[{'link': 'https://www.tbmm.gov.tr/tutanaklar/...,Erzincan İli Sosyal Yardımlaşma ve Dayanışma V...,32
1,[{'link': 'https://www.tbmm.gov.tr/tutanaklar/...,Sosyal Yardımlaşma ve Dayanışma Vakıfları tara...,32
161,[{'link': 'https://www.tbmm.gov.tr/tutanaklar/...,Sosyal Yardımlaşma ve Dayanışmayı Teşvik Fonun...,30
113,[{'link': 'https://www.tbmm.gov.tr/tutanaklar/...,Sosyal Yardımlaşma ve Dayanışmayı Teşvik Fonun...,30
27,[{'link': 'https://www.tbmm.gov.tr/tutanaklar/...,Sosyal yardımlaşma ve dayanışma vakıflarına il...,22
41,[{'link': 'https://www.tbmm.gov.tr/tutanaklar/...,Sosyal yardım yapan vakıf ve derneklere uygula...,22
